# cache

Commands and population data for importing in to a database.

Even M$ visual studio products have to cache symbols somewhere.

I suppose this notebook should offline documentation too.


## notebook cache

Information about the current library cached.


### exports cache

A cache of all exports, no matter the language.


#### the code

exports cache?


In [ ]:
// export cache automatically replaced

#### create export cache?



In [ ]:
var importer = require('../Core')
var getExports = importer.import('get exports from source')
var fs = require('fs')
var path = require('path')

function updateNotebook(exports) {
    var cache = {}
    exports.forEach(e => {
        cache[e.filename] = e.exports
    })
    console.log(cache)
    var code = `
// export cache automatically replaced
var exportCache = ${JSON.stringify(cache)}

module.exports = exportCache
`
    var cachePath = path.resolve(__dirname, './cache.ipynb')
    var notebook = JSON.parse(fs.readFileSync(cachePath))
    notebook.cells.forEach(c => {
        if(c.cell_type === 'code'
           && c.source.join('').includes('export cache automatically replaced')) {
            c.source = code.split('\n').map(line => line + '\n')
        }
    })
    fs.writeFileSync(cachePath, JSON.stringify(notebook, null, 4))
}

function promiseExports(cell) {
    return new Promise(resolve => {
        setTimeout(() => {
            try {
                if (process.memoryUsage().heapUsed > 200000000) {
                    throw new Error('out of memory')
                }
                cell.exports = getExports(cell.code)
            } catch (e) {
                cell.exports = e.message
            }
            resolve(cell)
        }, 100)
    })
}

function createExportCache() {
    var allCells = importer.interpret()
    var allCellIds = allCells.map(c => c.id)
    // get only first occurrence
    allCells = allCells
        .filter((c, i) => allCellIds.indexOf(c.id) == i)
        .filter(c => c.code.length < 10000
                && c.code.trim().length > 10)
    //    .slice(0, 10)
    return Promise
        .all(allCells.map(promiseExports))
        .then(updateNotebook)
}

module.exports = createExportCache

if(typeof $$ !== 'undefined') {
    createExportCache()
}


compiling /Users/briancullinan/jupyter_ops/Core/syntax.ipynb[2] aka ### get exports from source
compiling /Users/briancullinan/jupyter_ops/Core/syntax.ipynb[16] aka select ast
compiling /Users/briancullinan/jupyter_ops/Core/syntax.ipynb[11] aka tree to html
compiling /Users/briancullinan/jupyter_ops/Core/patterns.ipynb[2] aka ### select tree
compiling /Users/briancullinan/jupyter_ops/Core/patterns.ipynb[0] aka ,### walk tree


## TODO: bookmark cache

Always wanted to write something like evernote to automatically pull in the content of pages I bookmark. Incognito mode on news sites to avoid payment. Ad block plugins installed.  Might as well make ad-blocked cached copies of every page I visit.




## TODO: documentation cache

Tools for searching and downloading documention as it related to this library.

